In [1]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
import time
import random
import re

Начинаем с генерации последовательности

In [3]:
class Generator:
    def __init__(self, start_seq=1, iterations=1000):
        self.start = start_seq
        self.iter = iterations
        self.x_sequence = []
        self.y_sequence = []
        self.x = self.start
        self.y = self.x

    def generate_sequence(self) -> list | list:
        '''
        Функция генерирует 2 списка с последовательностями.
        Первый список по порядку, начало отсчёта start_seq.
        Второй список с условием:
        Y1 = X1
        Yi = Xi + X1
        если Yi >= 10, то Yi - 10
        '''
        self.x_sequence.append(self.x)
        self.y_sequence.append(self.y)
        while self.iter > 0:
            self.x += 1
            self.x_sequence.append(self.x)
            self.y = self.x + self.start
            if self.y >= 10:
                self.y = self.y - (self.y // 10) * 10
            self.y_sequence.append(self.y)
            self.iter -= 1
        return self.x_sequence, self.y_sequence


gen = Generator()
X, Y = gen.generate_sequence()

X_t = torch.tensor(X)
Y_t = torch.tensor(Y)

Отмечаем все слои модели

In [5]:
embed_len = len(X)
embed_size = 64
hidden_size = 128
out_size = 10

Небольшая функция, что бы вытянуть название модели, для отображения

In [7]:
def mod_name(string: str) -> str:
    '''
    Функция достаёт название модели
    '''
    prog = re.compile('[A-Z]{3,4}')
    res = prog.findall(string)
    return res[0]

Создаём класс модель

In [9]:
class Decode(torch.nn.Module):
    
    def __init__(self, RNNClass, embed_len, embed_size, hidden_size, out_size):
        super().__init__()
        self.embed = torch.nn.Embedding(embed_len, embed_size)
        self.rnn = RNNClass(embed_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, out_size)
        self.is_lstm = issubclass(RNNClass, torch.nn.LSTM)
        
    def forward(self, seq):
        embed = self.embed(seq)
        if self.is_lstm:
            all_layers, (_, _) = self.rnn(embed)
        else:
            all_layers, _ = self.rnn(embed)
        out = self.linear(all_layers)
        return out


Создаём класс для обучения модели

In [11]:
class SequenceModel:
    def __init__(self, model, step=10, epoch=100):
        super().__init__()
        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(model.parameters(), lr=.1)
        self.epoch = epoch
        self.step = step
        self.model = model
        self.model_name = mod_name(str(self.model.rnn))

    def train(self, X, Y):
        print(f'Начало обучения. Модель: {self.model_name}. Количество эпох {self.epoch}')
        train_time = time.time()
        for ep in range(self.epoch + 1):
            start = time.time()
            train_loss = 0.
            train_passed = 0
        
            for i in range(1000-self.step):
                X_batch = X[i: i+self.step]
                y_batch = Y[i: i+self.step]
                self.optimizer.zero_grad()
                answers = self.model.forward(X_batch)
                loss = self.criterion(answers, y_batch)
                train_loss += loss.item()
                
                loss.backward()
                self.optimizer.step()
                train_passed += 1
            if ep % 10 == 0 and ep != 0:
                print(f"Количество эпох: {ep}. Время на 10 эпох : {(time.time() - start) * 10:.3f}, loss текущей эпохи: {(train_loss / train_passed):.3f}")

        print(f'Общее время затраченное на обучение: {(time.time() - train_time):.2f} секунд')

Обучаем все три модели

In [13]:
rnn_model = SequenceModel(model=Decode(RNNClass=torch.nn.RNN, embed_len=embed_len, embed_size=embed_size, hidden_size=hidden_size, out_size=out_size))

In [14]:
rnn_model.train(X_t, Y_t)

Начало обучения. Модель: RNN. Количество эпох 100
Количество эпох: 10. Время на 10 эпох : 17.634, loss текущей эпохи: 0.011
Количество эпох: 20. Время на 10 эпох : 18.107, loss текущей эпохи: 0.003
Количество эпох: 30. Время на 10 эпох : 16.080, loss текущей эпохи: 0.002
Количество эпох: 40. Время на 10 эпох : 17.018, loss текущей эпохи: 0.001
Количество эпох: 50. Время на 10 эпох : 17.470, loss текущей эпохи: 0.001
Количество эпох: 60. Время на 10 эпох : 19.527, loss текущей эпохи: 0.001
Количество эпох: 70. Время на 10 эпох : 17.543, loss текущей эпохи: 0.001
Количество эпох: 80. Время на 10 эпох : 16.709, loss текущей эпохи: 0.001
Количество эпох: 90. Время на 10 эпох : 18.094, loss текущей эпохи: 0.000
Количество эпох: 100. Время на 10 эпох : 17.270, loss текущей эпохи: 0.000
Общее время затраченное на обучение: 169.33 секунд


In [15]:
lstm_model = SequenceModel(model=Decode(RNNClass=torch.nn.LSTM, embed_len=embed_len, embed_size=embed_size, hidden_size=hidden_size, out_size=out_size))

In [16]:
lstm_model.train(X_t, Y_t)

Начало обучения. Модель: LSTM. Количество эпох 100
Количество эпох: 10. Время на 10 эпох : 13.318, loss текущей эпохи: 0.017
Количество эпох: 20. Время на 10 эпох : 14.542, loss текущей эпохи: 0.005
Количество эпох: 30. Время на 10 эпох : 13.721, loss текущей эпохи: 0.003
Количество эпох: 40. Время на 10 эпох : 14.840, loss текущей эпохи: 0.002
Количество эпох: 50. Время на 10 эпох : 13.919, loss текущей эпохи: 0.002
Количество эпох: 60. Время на 10 эпох : 13.507, loss текущей эпохи: 0.001
Количество эпох: 70. Время на 10 эпох : 13.289, loss текущей эпохи: 0.001
Количество эпох: 80. Время на 10 эпох : 14.106, loss текущей эпохи: 0.001
Количество эпох: 90. Время на 10 эпох : 12.909, loss текущей эпохи: 0.001
Количество эпох: 100. Время на 10 эпох : 13.787, loss текущей эпохи: 0.001
Общее время затраченное на обучение: 139.16 секунд


In [17]:
gru_model = SequenceModel(model=Decode(RNNClass=torch.nn.GRU, embed_len=embed_len, embed_size=embed_size, hidden_size=hidden_size, out_size=out_size))

In [18]:
gru_model.train(X_t, Y_t)

Начало обучения. Модель: GRU. Количество эпох 100
Количество эпох: 10. Время на 10 эпох : 32.599, loss текущей эпохи: 0.010
Количество эпох: 20. Время на 10 эпох : 31.949, loss текущей эпохи: 0.003
Количество эпох: 30. Время на 10 эпох : 31.663, loss текущей эпохи: 0.002
Количество эпох: 40. Время на 10 эпох : 31.607, loss текущей эпохи: 0.001
Количество эпох: 50. Время на 10 эпох : 35.323, loss текущей эпохи: 0.001
Количество эпох: 60. Время на 10 эпох : 32.450, loss текущей эпохи: 0.001
Количество эпох: 70. Время на 10 эпох : 34.226, loss текущей эпохи: 0.001
Количество эпох: 80. Время на 10 эпох : 31.180, loss текущей эпохи: 0.001
Количество эпох: 90. Время на 10 эпох : 31.217, loss текущей эпохи: 0.001
Количество эпох: 100. Время на 10 эпох : 35.273, loss текущей эпохи: 0.000
Общее время затраченное на обучение: 333.16 секунд


Пишем функцию для проверки качества, для этого случайным образом отбираем числа из датасета, перемешиваем и подаём каждой модели на предсказание, складываем реальные значения и предсказанные в тензоры, после оцениваем качество предсказания.

In [20]:
class ModelEstimator():
    def __init__(self, model_list, X: list, Y: list):
        self.X, self.Y = X, Y
        self.model_list = model_list
        self.data = self.create_test_data(self.X, self.Y)

    def create_test_data(self, X: list, Y: list) -> 'Объект DataLoader':
        '''
        Функция принимает на вход две последовательности и возвращает
        объект DataLoader со случайной и перемешаной выборкой значений
        Инициализируется при создании экзепляра класса
        '''
        X_check = []
        Y_check = []
        for i in range(1000):
            choise = random.randint(1, 3)
            if choise > 2:
                X_check.append(X[i])
                Y_check.append(Y[i])
        dataset = TensorDataset(torch.tensor(X_check), torch.tensor(Y_check))
        dataloader = DataLoader(dataset, shuffle=True)
        return dataloader

    def estimate(self) -> 'NoReturn':
        '''
        Функция проверяет точноть моделей по соответствию значений реальных и предсказанных
        '''
        for model in self.model_list:
            true_value = []
            predictd_value = []
            for x, y in self.data:
                predictd_value.append(torch.argmax(rnn_model.model(torch.tensor([x]))))
                true_value.append(y.squeeze())
            acc = torch.sum(torch.eq(torch.tensor(predictd_value), torch.tensor(true_value))).item()/torch.tensor(true_value).nelement()
            print(f'Точность модели {model.model_name}: {acc:.2f}')
            

In [21]:
me = ModelEstimator([rnn_model, lstm_model, gru_model], X, Y)
me.estimate()

Точность модели RNN: 0.99
Точность модели LSTM: 0.99
Точность модели GRU: 0.99
